# Table Reflection

Sebuah objek Table bisa diinstruksikan to memuat informasi mengenai dirinya sendiri dari dari objek skema database terkait yang sudah ada di dalam database, dan proses tersebut disebut <b>reflection</b>. Table reflection mengacu pada proses pembuatan table dan objek yang terkait dengan membaca status database saat ini. Berikut ini adalah contoh dari table reflection:

<b>some_table = Table("some_table", metadata, autoload=engine)</b>

Pada contoh di bawah ini akan memcoba membuat tabel dari database SQLite yang sudah ada yang di dalamnya sudah terdapat dua buah tabel yaitu tabel users dan tabel email. 

In [4]:
from sqlalchemy import create_engine, MetaData, Table

engine = create_engine("sqlite+pysqlite:///example.db", future=True)
metadata = MetaData(bind=engine)

In [5]:
metadata.tables

FacadeDict({})

Metadata belum berisi apa-apa, selanjutnya metadata tersebut akan diisi dengan tabel

## Membuat tabel email

In [6]:
email_table = Table("email", metadata, autoload=engine)

In [7]:
metadata.tables

FacadeDict({'email': Table('email', MetaData(bind=Engine(sqlite+pysqlite:///example.db)), Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False), Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>, nullable=False), Column('email_address', VARCHAR(), table=<email>, nullable=False), schema=None), 'users': Table('users', MetaData(bind=Engine(sqlite+pysqlite:///example.db)), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('name', VARCHAR(length=30), table=<users>), Column('fullname', VARCHAR(), table=<users>), schema=None)})

Sekarang metadata sudah berisi informasi mengenai tabel yang baru saja dibuat.

In [5]:
email_table

Table('email', MetaData(bind=Engine(sqlite+pysqlite:///example.db)), Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False), Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>, nullable=False), Column('email_address', VARCHAR(), table=<email>, nullable=False), schema=None)

Tabel email menginformasikan bahwa tabel email ini mempunyai relasi dengan tabel users karena kolom user_id pada tabel email merupakan foreign key untuk kolom id pada tabel users.

In [11]:
email_table.c.items()

[('id',
  Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False)),
 ('user_id',
  Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>, nullable=False)),
 ('email_address',
  Column('email_address', VARCHAR(), table=<email>, nullable=False))]

## Membuat tabel users

In [12]:
users_table = Table("users", metadata, autoload=engine)

In [14]:
users_table

Table('users', MetaData(bind=Engine(sqlite+pysqlite:///example.db)), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('name', VARCHAR(length=30), table=<users>), Column('fullname', VARCHAR(), table=<users>), schema=None)

In [15]:
users_table.c.items()

[('id',
  Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False)),
 ('name', Column('name', VARCHAR(length=30), table=<users>)),
 ('fullname', Column('fullname', VARCHAR(), table=<users>))]

In [24]:
metadata.sorted_tables

[Table('users', MetaData(bind=Engine(sqlite+pysqlite:///example.db)), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('name', VARCHAR(length=30), table=<users>), Column('fullname', VARCHAR(), table=<users>), schema=None),
 Table('email', MetaData(bind=Engine(sqlite+pysqlite:///example.db)), Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False), Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>, nullable=False), Column('email_address', VARCHAR(), table=<email>, nullable=False), schema=None)]

In [8]:
"users" in metadata.tables

True

## Reflecting All Tables at Once

In [19]:
from sqlalchemy import create_engine, MetaData, Table

engine = create_engine("sqlite+pysqlite:///example.db", future=True)

In [20]:
metadata = MetaData()

In [21]:
metadata.reflect(bind=engine)

In [22]:
metadata.tables

FacadeDict({'email': Table('email', MetaData(), Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False), Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>, nullable=False), Column('email_address', VARCHAR(), table=<email>, nullable=False), schema=None), 'users': Table('users', MetaData(), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('name', VARCHAR(length=30), table=<users>), Column('fullname', VARCHAR(), table=<users>), schema=None)})

In [23]:
for table in metadata.tables:
    print(table)

email
users


In [24]:
email_table_copy = metadata.tables["email"]
users_table_copy = metadata.tables["users"]

In [25]:
email_table_copy.c.items()

[('id',
  Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False)),
 ('user_id',
  Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>, nullable=False)),
 ('email_address',
  Column('email_address', VARCHAR(), table=<email>, nullable=False))]

In [26]:
users_table_copy.c.items()

[('id',
  Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False)),
 ('name', Column('name', VARCHAR(length=30), table=<users>)),
 ('fullname', Column('fullname', VARCHAR(), table=<users>))]

## Reflection with Inspector

In [1]:
from sqlalchemy import create_engine, MetaData, Table, inspect

engine = create_engine("sqlite+pysqlite:///example.db", future=True)

In [9]:
insp = inspect(engine)

In [22]:
insp.dialect

In [12]:
insp.engine

Engine(sqlite+pysqlite:///example.db)

In [20]:
insp.get_table_names()

['email', 'users']

In [23]:
insp.get_columns("email")

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'user_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'email_address',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

## Mapping Declaratively with Reflected Tables

In [1]:
from sqlalchemy import create_engine, Table
from sqlalchemy.orm import declarative_base

In [2]:
engine = create_engine("sqlite+pysqlite:///example2.db", future=True)
Base = declarative_base(bind=engine)

In [3]:
class Users(Base):
    __table__ = Table("users", Base.metadata, autoload=engine)
    __tablename__ = "users"
    
    def __repr__(self):
        return f'User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})'

In [4]:
Users

__main__.Users

In [5]:
Users.__table__

Table('users', MetaData(bind=Engine(sqlite+pysqlite:///example2.db)), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('name', VARCHAR(length=30), table=<users>), Column('fullname', VARCHAR(), table=<users>), schema=None)

In [6]:
Users.__tablename__

'users'

In [7]:
muhlisadiwiguna = Users(name="Muhlis", fullname="Muhlis Adiwiguna")

In [8]:
muhlisadiwiguna

User(id=None, name='Muhlis', fullname='Muhlis Adiwiguna')

In [9]:
class Email(Base):
    __table__ = Table("email", Base.metadata, autoload=engine)
    __tablename__ = "email"
    
    def __repr__(self):
        return f'Email(id={self.id!r}, email_address={self.email_address!r})'

In [10]:
Email.__table__

Table('email', MetaData(bind=Engine(sqlite+pysqlite:///example2.db)), Column('id', INTEGER(), table=<email>, primary_key=True, nullable=False), Column('email_address', VARCHAR(), table=<email>, nullable=False), Column('user_id', INTEGER(), ForeignKey('users.id'), table=<email>), schema=None)

In [11]:
Email.__tablename__

'email'